# 숙제1: ETL함수 작성
Extract, Transform(Data format 변환), Load(redshift 테이블에 푸쉬) 

1. INSERT INTO SQL 완성하기
2. 기존 테이블의 내용을 지우고 새로 업데이트 하는 방식 Full-Refresh라면 어떻게 하는게 좋을지 (트랜잭션을 생각, Full-Refresh형태로 동작하게 load 함수 변경 필요)


In [ ]:
import requests
def extract(url):
  f = requests.get(url)
  return (f.text)

In [ ]:

import csv
def transform(date):
  reader = csv.reader(data.strip().split("\n"))
  return reader

In [ ]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection():
    host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
    redshift_user = "ID"
    redshift_pass = "PW"
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
        dbname=dbname,
        user=redshift_user,
        password=redshift_pass,
        host=host,
        port=port
    ))
    conn.set_session(autocommit=True)
    return conn.cursor()
  


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
def load(csv_reader):
    cur = get_Redshift_connection()
 
    for name, gender in csv_reader:
        print(name, gender)
        sql = "INSERT INTO raw_data.name_gender VALUES (name, gender)" 
    cur.execute(sql)


Full-Refresh 형태로 만들기

In [ ]:
# 내답안

def load(csv_reader):
    cur = get_Redshift_connection()
    sql = """DROP TABLE IF EXISTS raw_data.name_gender \
        CREATE TABLE raw_data.name_gender;"""
    #TRUNCATE는 Transaction 안에서는 쓰면 안됨. Delete from 은 Rollback하지만 truncate은 못함.
    
  
    cur.execute(sql)
    for name, gender in csv_reader:
       
        sql = "INSERT INTO raw_data.name_gender VALUES ({name}, {gender})".format(name = name, gender = gender)
    cur.execute(sql)    

In [ ]:
#Solution 1 - Begin / End
def load(csv_reader):
    cur = get_Redshift_connection() 
    # cur: autocommit = True - BEGIN;END
    sql = """BEGIN;CREATE TALBE IF NOT EXISTS raw_data.name_gender (
            name varchar(64),
            gender varchar(16)
            );
            DELETE FROM raw_data.name_gender;"""
    cur.execute(sql)   
    for name, gender in csv_reader:
        sql = """
            INSERT INTO raw_data.name_gender
            VALUES ({name}, {gender});
            """.format(name=name, gender=gender)
        cur.execute(sql)
    cur.execute("END")


In [ ]:
#Solution 2 - Try/catch
#cur : autocommit = True

def load(csv_reader):
    cur = get_Redshift_connection() 
    try:
      sql = """BEGIN;CREATE TALBE IF NOT EXISTS raw_data.name_gender (
              name varchar(64),
              gender varchar(16)
              );
              DELETE FROM raw_data.name_gender;"""
      cur.execute(sql)   
      for name, gender in csv_reader:
          sql = """
              INSERT INTO raw_data.name_gender
              VALUES ({name}, {gender});
              """.format(name=name, gender=gender)
          cur.execute(sql)
      cur.execute("END") # cur.execute("COMMIT")
    except Exception as e:
      cur.execute("ROLLBACK")    #error나면 지금까지 했던거 다 무시하고 begin전으로 돌아가라



#autocommit = False:
def load(csv_reader):
    cur = get_Redshift_connection() 
    try:
      sql = """CREATE TALBE IF NOT EXISTS raw_data.name_gender (
              name varchar(64),
              gender varchar(16)
              );
              DELETE FROM raw_data.name_gender;"""
      cur.execute(sql)   
      for name, gender in csv_reader:
          sql = """
              INSERT INTO raw_data.name_gender
              VALUES ({name}, {gender});
              """.format(name=name, gender=gender)
          cur.execute(sql)
      cur.execute("END") # cur.execute("COMMIT")
    except Exception as e:
      cur.execute("ROLLBACK") 


#숙제 2: Open Weather Map API 
1. API Key를 받아서 서울의 위도 경도 찾을것
2. 서울의 정보 (날짜, 평균온도, 최소온도, 최대온도)만 앞으로 7일 대상으로 출력
https://api.openweathermap.org/data/2.5/onecall?lat={37.56}&lon{126.97}&appid={00a33e28fdb3098bec6c02b7c229ffe8}

## 1) 서울의 위도, 경도 찾기

In [ ]:
import requests, json
api_key = "00a33e28fdb3098bec6c02b7c229ffe8"
city_name = "Seoul"
lon = 0
lat = 0
base_url = "http://api.openweathermap.org/data/2.5/weather?"
complete_url = base_url + "appid=" + api_key + "&q=" + city_name
response = requests.get(complete_url)
x = response.json()

if x['cod'] != '404':
  y = x['main']
  lon = x['coord']['lon']
  lat = x['coord']['lat']
  print(f"longtitude: {lon} \n latitude: {lat}")
else: ("Seoul Not Found")

longtitude: 126.9778 
 latitude: 37.5683


##2) 위도 경도 정보로 서울의 향후 7일 데이타 찾기


In [ ]:
api_key = "00a33e28fdb3098bec6c02b7c229ffe8"
#not work !! url = "https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&appid={'00a33e28fdb3098bec6c02b7c229ffe8'}"
url ='https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude={}&units={}&appid={}'.format(37.57,126.98,'current,minutely,hourly','metric',"00a33e28fdb3098bec6c02b7c229ffe8")

response = requests.get(url)
data = response.json()
print(data)
print(data['daily'])

import pandas as pd
df = pd.json_normalize(data, record_path = ['daily'])
print(df)

{'lat': 37.57, 'lon': 126.98, 'timezone': 'Asia/Seoul', 'timezone_offset': 32400, 'daily': [{'dt': 1634958000, 'sunrise': 1634939250, 'sunset': 1634978705, 'moonrise': 1634984100, 'moonset': 1634947260, 'moon_phase': 0.58, 'temp': {'day': 12.08, 'min': 9.39, 'max': 17.31, 'night': 12.61, 'eve': 14.94, 'morn': 9.79}, 'feels_like': {'day': 10.83, 'night': 11, 'eve': 13.3, 'morn': 9.79}, 'pressure': 1029, 'humidity': 57, 'dew_point': 3.83, 'wind_speed': 3.4, 'wind_deg': 267, 'wind_gust': 3.35, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': 0, 'pop': 0, 'uvi': 3.85}, {'dt': 1635044400, 'sunrise': 1635025709, 'sunset': 1635065030, 'moonrise': 1635072600, 'moonset': 1635037200, 'moon_phase': 0.61, 'temp': {'day': 15.87, 'min': 9.99, 'max': 16.98, 'night': 12.71, 'eve': 15.26, 'morn': 10.3}, 'feels_like': {'day': 14.48, 'night': 11.24, 'eve': 13.6, 'morn': 8.64}, 'pressure': 1028, 'humidity': 37, 'dew_point': 0.92, 'wind_speed': 2.65, 'wind_deg

In [ ]:
# epoch 을 datetime형태로 변환
from datetime import *
newdt=[]

for date in df['dt'] :
  newdt.append(datetime.fromtimestamp(date).strftime('%Y-%m-%d'))

df['dt'] = newdt


df.head()


,dt,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,weather,clouds,pop,uvi,temp.day,temp.min,temp.max,temp.night,temp.eve,temp.morn,feels_like.day,feels_like.night,feels_like.eve,feels_like.morn,rain
0,2021-10-23,1634939250,1634978705,1634984100,1634947260,0.58,1029,57,3.83,3.40,267,3.35,"[{'id': 800, 'main': 'Clear', 'description': '...",0,0.00,3.85,12.08,9.39,17.31,12.61,14.94,9.79,10.83,11.00,13.30,9.79,NaN
1,2021-10-24,1635025709,1635065030,1635072600,1635037200,0.61,1028,37,0.92,2.65,299,3.39,"[{'id': 800, 'main': 'Clear', 'description': '...",0,0.00,3.65,15.87,9.99,16.98,12.71,15.26,10.30,14.48,11.24,13.60,8.64,NaN
2,2021-10-25,1635112170,1635151356,1635161400,1635127080,0.64,1024,38,1.49,2.97,308,3.70,"[{'id': 800, 'main': 'Clear', 'description': '...",0,0.00,3.42,15.93,10.16,17.42,13.34,15.70,10.40,14.57,12.19,14.40,8.85,NaN
3,2021-10-26,1635198630,1635237683,1635250620,1635216840,0.67,1021,51,6.35,2.71,274,3.01,"[{'id': 804, 'main': 'Clouds', 'description': ...",100,0.00,3.17,16.74,11.76,18.15,15.33,16.24,11.76,15.80,14.62,15.46,10.71,NaN
4,2021-10-27,1635285091,1635324011,1635340140,1635306300,0.70,1018,54,7.99,4.16,289,7.32,"[{'id': 500, 'main': 'Rain', 'description': 'l...",2,0.63,3.31,17.57,12.81,18.56,13.98,16.74,12.81,16.79,12.56,15.70,12.39,2.0


### Dataframe에 날짜, 최소온도, 최대온도, 평균온도만

In [ ]:
colname = ['dt','temp.min', 'temp.max', 'temp.day']
df = df.drop(columns=[col for col in df if col not in colname])
# 평균온도가 min max의 평균일때 : df['AVG_Temp'] = df[['temp.min', 'temp.max']].mean(numeric_only=True, axis=1).round(2)
df = df.rename(columns={'dt': 'Date', 'temp.day':'AVG_Temp','temp.min': 'Min_Temp', 'temp.max': 'Max_Temp'})
df.head(10)

,Date,AVG_Temp,Min_Temp,Max_Temp
0,2021-10-23,12.08,9.39,17.31
1,2021-10-24,15.87,9.99,16.98
2,2021-10-25,15.93,10.16,17.42
3,2021-10-26,16.74,11.76,18.15
4,2021-10-27,17.57,12.81,18.56
5,2021-10-28,16.33,11.18,17.79
6,2021-10-29,17.17,11.06,18.79
7,2021-10-30,17.15,12.08,17.93


In [ ]:
# Solution:
daily = data["daily"]

from datetime import *

"""
 멱등성: 1번 실행하나 10번 연달아 실행하나 같은 레코드들을 갖고 있어야 한다
CREATE TABLE keeyong.seoul_temp (
    date DATE PRIMARY KEY,
    day float,
    min float,
    max float,
    created timestamp DEFAULT current_date()
);

ROW_NUMBER() OVER (PARTITION BY date ORDER BY created DESC) nn
"""

for entry in daily:
    dt = datetime.fromtimestamp(entry["dt"]).strftime('%Y-%m-%d') #날짜
    day = entry["temp"]["day"] #평균온도
    min = entry["temp"]["min"] #최소온도 
    max = entry["temp"]["max"] #최대온도
    print(f"{dt:11}{day:7}{min:7}{max:7}")

2021-10-23   12.08   9.39  17.31
2021-10-24   15.87   9.99  16.98
2021-10-25   15.93  10.16  17.42
2021-10-26   16.74  11.76  18.15
2021-10-27   17.57  12.81  18.56
2021-10-28   16.33  11.18  17.79
2021-10-29   17.17  11.06  18.79
2021-10-30   17.15  12.08  17.93
